In [1]:
import pandas as pd
from snowflake.snowpark.session import Session
from snowflake.snowpark.functions import udf, avg, col
from snowflake.snowpark.types import IntegerType, FloatType, StringType, BooleanType
from snowflake.snowpark.files import SnowflakeFile
import sys
sys.path.append('..')
from credentials import Credentials

In [ ]:
# Setting pandas options
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [2]:
cred = Credentials()
session = Session.builder.configs(cred.__dict__).create()

In [5]:
session.use_role("SYSADMIN")
session.use_database("STITCH")
session.use_warehouse("ACCOUNTADMIN_MGMT")
session.use_schema("SALESFORCEFSL3")

In [6]:
# Getting name of the table
db_name = 'STITCH'
schema_name = 'SALESFORCEFSL3'
table_name:str = 'ACCOUNT'

In [8]:
result = session.sql(f"DESCRIBE TABLE {db_name}.{schema_name}.{table_name};".format(db_name, schema_name, table_name)).collect()

In [14]:
# convert all the row elements to dictionaries
result = [row.as_dict() for row in result]

In [15]:
df = pd.DataFrame(result)

In [16]:
df

,name,type,kind,null?,default,primary key,unique key,check,expression,comment,policy name
0,ABM_TARGET_ACCOUNT__C,BOOLEAN,COLUMN,Y,None,N,N,None,None,None,None
1,ACCOUNTNUMBER,VARCHAR(16777216),COLUMN,Y,None,N,N,None,None,None,None
2,ACCOUNT_ISACTIVE_CURRENT_FY__C,BOOLEAN,COLUMN,Y,None,N,N,None,None,None,None
3,ACCOUNT_ISACTIVE_CURRENT_PREVIOUS_FY__C,BOOLEAN,COLUMN,Y,None,N,N,None,None,None,None
4,ACCOUNT_ISACTIVE_LAST_12_MONTHS__C,BOOLEAN,COLUMN,Y,None,N,N,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...
104,_SDC_RECEIVED_AT,TIMESTAMP_TZ(9),COLUMN,Y,None,N,N,None,None,None,None
105,_SDC_SEQUENCE,"NUMBER(38,0)",COLUMN,Y,None,N,N,None,None,None,None
106,_SDC_TABLE_VERSION,"NUMBER(38,0)",COLUMN,Y,None,N,N,None,None,None,None
107,LASTREFERENCEDDATE,TIMESTAMP_TZ(9),COLUMN,Y,None,N,N,None,None,None,None


In [ ]:
# a lambda function that creates a new column called 'name_new' if the type is like VARCHAR then add a "000" to the origial name else just add the original name
# df['name_new'] = df.apply(lambda x: x['name'] + '000' if isinstance(x['type'], str) and x['type'].find('VARCHAR') != -1 else x['name'], axis=1)

In [55]:
def transform_name(name, type):
    if isinstance(type, str) and 'VARCHAR' in type:
        #return name + '0007'
        return f"CAST(SUBSTR({name},1,10000) AS VARCHAR(10000)) AS {name}".format(name)
    else:
        return f"{name} AS {name}".format(name)

# apply the transform_name function to create a new column called 'name_new'
df['name_new'] = df.apply(lambda x: transform_name(x['name'], x['type']), axis=1)

In [59]:

#df[['name','type','name_new']].head(200)
column_list:list = df['name_new'].to_list()

In [61]:
columns_str:str = ', '.join(column_list)

In [62]:
columns_str

'ABM_TARGET_ACCOUNT__C AS ABM_TARGET_ACCOUNT__C, CAST(SUBSTR(ACCOUNTNUMBER,1,10000) AS VARCHAR(10000)) AS ACCOUNTNUMBER, ACCOUNT_ISACTIVE_CURRENT_FY__C AS ACCOUNT_ISACTIVE_CURRENT_FY__C, ACCOUNT_ISACTIVE_CURRENT_PREVIOUS_FY__C AS ACCOUNT_ISACTIVE_CURRENT_PREVIOUS_FY__C, ACCOUNT_ISACTIVE_LAST_12_MONTHS__C AS ACCOUNT_ISACTIVE_LAST_12_MONTHS__C, CAST(SUBSTR(ACCOUNT_OWNER_ROLE__C,1,10000) AS VARCHAR(10000)) AS ACCOUNT_OWNER_ROLE__C, ACCOUNT_PLAN_COUNT__C AS ACCOUNT_PLAN_COUNT__C, CAST(SUBSTR(ACCOUNT_RECORD_ID__C,1,10000) AS VARCHAR(10000)) AS ACCOUNT_RECORD_ID__C, ACCOUNT_REGION__C AS ACCOUNT_REGION__C, CAST(SUBSTR(AM_IS_ME__C,1,10000) AS VARCHAR(10000)) AS AM_IS_ME__C, CAST(SUBSTR(BILLINGCITY,1,10000) AS VARCHAR(10000)) AS BILLINGCITY, CAST(SUBSTR(BILLINGCOUNTRY,1,10000) AS VARCHAR(10000)) AS BILLINGCOUNTRY, CAST(SUBSTR(BILLINGCOUNTRYCODE,1,10000) AS VARCHAR(10000)) AS BILLINGCOUNTRYCODE, CAST(SUBSTR(BILLINGGEOCODEACCURACY,1,10000) AS VARCHAR(10000)) AS BILLINGGEOCODEACCURACY, BILLINGLATI

In [63]:
# append the final result to something like "CREATE OR REPLACE VIEW {db_name}.{schema_name}}.{table_name}_V AS SELECT {columns_str} FROM {db_name}.{schema_name}.{table_name};"
final_query:str = f"CREATE OR REPLACE VIEW {db_name}.{schema_name}.{table_name}_V AS SELECT {columns_str} FROM {db_name}.{schema_name}.{table_name};"

In [64]:
final_query

'CREATE OR REPLACE VIEW STITCH.SALESFORCEFSL3.ACCOUNT_V AS SELECT ABM_TARGET_ACCOUNT__C AS ABM_TARGET_ACCOUNT__C, CAST(SUBSTR(ACCOUNTNUMBER,1,10000) AS VARCHAR(10000)) AS ACCOUNTNUMBER, ACCOUNT_ISACTIVE_CURRENT_FY__C AS ACCOUNT_ISACTIVE_CURRENT_FY__C, ACCOUNT_ISACTIVE_CURRENT_PREVIOUS_FY__C AS ACCOUNT_ISACTIVE_CURRENT_PREVIOUS_FY__C, ACCOUNT_ISACTIVE_LAST_12_MONTHS__C AS ACCOUNT_ISACTIVE_LAST_12_MONTHS__C, CAST(SUBSTR(ACCOUNT_OWNER_ROLE__C,1,10000) AS VARCHAR(10000)) AS ACCOUNT_OWNER_ROLE__C, ACCOUNT_PLAN_COUNT__C AS ACCOUNT_PLAN_COUNT__C, CAST(SUBSTR(ACCOUNT_RECORD_ID__C,1,10000) AS VARCHAR(10000)) AS ACCOUNT_RECORD_ID__C, ACCOUNT_REGION__C AS ACCOUNT_REGION__C, CAST(SUBSTR(AM_IS_ME__C,1,10000) AS VARCHAR(10000)) AS AM_IS_ME__C, CAST(SUBSTR(BILLINGCITY,1,10000) AS VARCHAR(10000)) AS BILLINGCITY, CAST(SUBSTR(BILLINGCOUNTRY,1,10000) AS VARCHAR(10000)) AS BILLINGCOUNTRY, CAST(SUBSTR(BILLINGCOUNTRYCODE,1,10000) AS VARCHAR(10000)) AS BILLINGCOUNTRYCODE, CAST(SUBSTR(BILLINGGEOCODEACCURACY,1

In [66]:
session.sql(final_query).collect()

[Row(status='View ACCOUNT_V successfully created.')]